In [24]:
# !pip install spacy
import spacy
# python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")
import pandas as pd
import sklearn.linear_model as lm
import sklearn.svm as svm

In [26]:
triggers = pd.read_csv("handwritten_triggers.csv", header=None)[0]

In [30]:
context = pd.read_csv("handwritten_context.csv", header=None)[0]

In [83]:
negatives = pd.read_csv("samples.txt", header=None, delimiter='\n')[0]
negative_paragraphs = []
for i in range(len(negatives) // 5):
    negative_paragraphs.append(negatives[i])
    for j in range(1, 5):
        negative_paragraphs[-1] += negatives[i+j]
negative_paragraphs = pd.Series(negative_paragraphs)

In [84]:
def avg_similarity_with_triggers(doc):
    tot = 0
    for trigger in triggers:
        tot += doc.similarity(nlp(trigger))
    return tot / len(triggers)

In [85]:
contexts_avg_similarities = context.apply(nlp).apply(avg_similarity_with_triggers)

In [86]:
# shuffle
negative_paragraphs = negative_paragraphs.sample(len(negative_paragraphs))

In [87]:
data = pd.DataFrame()
data['score'] = contexts_avg_similarities
data['label'] = [1] * len(triggers_avg_similarities)

In [89]:
# only using 30 random negative sentences to have as label of 0
negative_data = pd.DataFrame()
negative_data['score'] = negative_paragraphs[:30].apply(nlp).apply(avg_similarity_with_triggers)
negative_data['label'] = [0] * 30

data = data.append(negative_data)

In [95]:
# try out a very basic SVM to create a linear boundary
model = svm.LinearSVC()

In [ ]:
model.fit(data[['score']], data['label'])

In [100]:
# Accuracy on training data
model.score(data[['score']], data['label'])

0.8666666666666667

In [98]:
model.predict([[avg_similarity_with_triggers(nlp("what is the weather today ?"))]])

array([1], dtype=int64)

In [103]:
model.predict([[avg_similarity_with_triggers(nlp("The incoming storm has been destroying everything in its path. We should get out of Florida before it heads in this direction. We should be able to pack up and move to the house in the west. Hopefully, they have better weather there."))]])

array([1], dtype=int64)